In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import *
from tqdm import tqdm

In [2]:
df_jounrey = pd.read_csv("data/df_journey.csv")

In [3]:
df_stop = pd.read_csv("save/stop_times.txt")
df_station = pd.read_csv("save/stops.txt")

In [4]:
df_trip = pd.read_csv("save/trips.txt")
df_route = pd.read_csv("save/routes.txt")

In [5]:
df_shape = pd.read_csv("save/shapes.txt")

In [6]:
df_calendar = pd.read_csv("save/calendar_dates.txt")
service_id_11_05 = df_calendar[(df_calendar["date"] == 20230511)]["service_id"].unique()
service_id_12_05 = df_calendar[(df_calendar["date"] == 20230512)]["service_id"].unique()

In [7]:
df_station[df_station["stop_name"]=="Globen"]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,platform_code
333,9021001001611000,Globen,59.294247,18.078015,1,NaN,NaN
433,9021001004503000,Globen,59.294124,18.076611,1,NaN,NaN
7544,9022001001611001,Globen,59.294243,18.078006,0,9.021001e+15,1
7545,9022001001611002,Globen,59.294252,18.078024,0,9.021001e+15,2
7748,9022001004503001,Globen,59.294033,18.076131,0,9.021001e+15,3
7749,9022001004503002,Globen,59.294214,18.077091,0,9.021001e+15,4


In [8]:
df_trip

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,9011001000100000,1,14010000574102918,NaN,1,1014010000482329256
1,9011001000100000,1,14010000574102788,NaN,1,1014010000482329256
2,9011001000100000,1,14010000574107201,NaN,1,1014010000482329256
3,9011001000100000,1,14010000574102742,NaN,1,1014010000482329256
4,9011001000100000,1,14010000574102962,NaN,1,1014010000482329256
...,...,...,...,...,...,...
106588,9011008004000000,301,14010000632876365,NaN,0,6014010000632876048
106589,9011008004000000,165,14010000632876477,NaN,0,6014010000632876048
106590,9011008004000000,1074,14010000632876534,NaN,0,6014010000632876048
106591,9011008004000000,208,14010000632876421,NaN,0,6014010000632876048


In [9]:
df_stop

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,14010000574102918,07:38:00,07:38:00,9022001010028003,1,Stora Essingen,3,1,0.00,1
1,14010000574102918,07:38:45,07:38:45,9022001010052003,2,Stora Essingen,3,3,398.26,0
2,14010000574102918,07:39:06,07:39:06,9022001010054002,3,Stora Essingen,3,3,570.67,0
3,14010000574102918,07:39:34,07:39:34,9022001010056002,4,Stora Essingen,3,3,808.22,0
4,14010000574102918,07:40:12,07:40:12,9022001010058002,5,Stora Essingen,3,3,1107.58,0
...,...,...,...,...,...,...,...,...,...,...
2090774,14010000632876421,19:15:00,19:15:00,9022001000534001,15,Norrtälje/Arholma (NordSydlinjen) via Furusund,3,3,NaN,1
2090775,14010000632876421,19:25:00,19:25:00,9022001000552001,16,Norrtälje/Arholma (NordSydlinjen) via Furusund,3,3,NaN,1
2090776,14010000632876421,20:15:00,20:15:00,9022001000554001,17,Norrtälje/Arholma (NordSydlinjen) via Furusund,1,3,NaN,1
2090777,14010000631915168,15:30:00,15:30:00,9022001000541001,1,Räfsnäs,1,1,NaN,1


In [10]:
def get_route_id_from_line(name_line: str,df_route: pd.DataFrame):
    return df_route[(df_route["route_short_name"]==name_line)&(df_route["route_desc"]!="Waxholmsbolagets")]["route_id"].values[0]

def get_trip_id_from_line(name_line: str,df_trip: pd.DataFrame,df_route: pd.DataFrame):
    return df_trip[df_trip["route_id"]==get_route_id_from_line(name_line,df_route)]["trip_id"].unique()

def get_trip_id_from_line_in_service_ids(name_line: str,df_trip: pd.DataFrame,df_route: pd.DataFrame, service_ids: np.ndarray):
    return df_trip[(df_trip["route_id"]==get_route_id_from_line(name_line,df_route))&(df_trip["service_id"].isin(service_ids))]["trip_id"].unique()

In [11]:
lines_interest = np.array(['14', '4', '11', '10', '17', '18', '19', '30', '177', '176'])

In [12]:
trip_test = get_trip_id_from_line("30",df_trip,df_route)[0]

In [13]:
df_stop[df_stop["trip_id"]==trip_test]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
352226,14010000606032112,06:18:00,06:18:00,9022001004525001,1,Solna station,3,1,0.00,1
352227,14010000606032112,06:21:00,06:21:00,9022001004535001,2,Solna station,3,3,1283.72,1
352228,14010000606032112,06:23:00,06:23:00,9022001004537001,3,Solna station,3,3,2183.49,1
352229,14010000606032112,06:25:00,06:25:00,9022001004539001,4,Solna station,3,3,2916.35,1
352230,14010000606032112,06:28:00,06:28:00,9022001004541001,5,Solna station,3,3,3529.09,1
352231,14010000606032112,06:29:30,06:29:30,9022001004543001,6,Solna station,3,3,3992.05,1
352232,14010000606032112,06:31:00,06:31:00,9022001004545001,7,Solna station,3,3,4648.40,1
352233,14010000606032112,06:33:00,06:33:00,9022001004547001,8,Solna station,3,3,5818.17,1
352234,14010000606032112,06:37:00,06:37:00,9022001004549002,9,Solna station,1,3,6607.81,1


In [14]:
df_stop

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,14010000574102918,07:38:00,07:38:00,9022001010028003,1,Stora Essingen,3,1,0.00,1
1,14010000574102918,07:38:45,07:38:45,9022001010052003,2,Stora Essingen,3,3,398.26,0
2,14010000574102918,07:39:06,07:39:06,9022001010054002,3,Stora Essingen,3,3,570.67,0
3,14010000574102918,07:39:34,07:39:34,9022001010056002,4,Stora Essingen,3,3,808.22,0
4,14010000574102918,07:40:12,07:40:12,9022001010058002,5,Stora Essingen,3,3,1107.58,0
...,...,...,...,...,...,...,...,...,...,...
2090774,14010000632876421,19:15:00,19:15:00,9022001000534001,15,Norrtälje/Arholma (NordSydlinjen) via Furusund,3,3,NaN,1
2090775,14010000632876421,19:25:00,19:25:00,9022001000552001,16,Norrtälje/Arholma (NordSydlinjen) via Furusund,3,3,NaN,1
2090776,14010000632876421,20:15:00,20:15:00,9022001000554001,17,Norrtälje/Arholma (NordSydlinjen) via Furusund,1,3,NaN,1
2090777,14010000631915168,15:30:00,15:30:00,9022001000541001,1,Räfsnäs,1,1,NaN,1


In [16]:
df_stop["stop_name"] = ""
df_stop["line_name"] = ""
hours = df_stop["arrival_time"].apply(lambda x : int(x[:2]))
service_id_day = [service_id_11_05,service_id_12_05]
hour_min_day = [14,5]
hour_max_day = [20,11]
for i in range(len(service_id_day)):
    for line in tqdm(lines_interest):
        id_trips = get_trip_id_from_line_in_service_ids(line,df_trip,df_route,service_id_day[i])
        for id_trip in id_trips:
            df_stop.loc[(df_stop["trip_id"]==id_trip)&(hours>hour_min_day[i])&(hours<hour_max_day[i]),"line_name"] = line
            df_stop.loc[(df_stop["trip_id"]==id_trip)&(hours>hour_min_day[i])&(hours<hour_max_day[i]),"stop_name"] = df_stop["stop_id"][(df_stop["trip_id"]==id_trip)&(hours>hour_min_day[i])&(hours<hour_max_day[i])].apply(lambda x : df_station[df_station["stop_id"]==x]["stop_name"].values[0])

100%|██████████| 10/10 [01:43<00:00, 10.31s/it]


In [17]:
df_stop

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,stop_name,line_name
0,14010000574102918,07:38:00,07:38:00,9022001010028003,1,Stora Essingen,3,1,0.00,1,,
1,14010000574102918,07:38:45,07:38:45,9022001010052003,2,Stora Essingen,3,3,398.26,0,,
2,14010000574102918,07:39:06,07:39:06,9022001010054002,3,Stora Essingen,3,3,570.67,0,,
3,14010000574102918,07:39:34,07:39:34,9022001010056002,4,Stora Essingen,3,3,808.22,0,,
4,14010000574102918,07:40:12,07:40:12,9022001010058002,5,Stora Essingen,3,3,1107.58,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
2090774,14010000632876421,19:15:00,19:15:00,9022001000534001,15,Norrtälje/Arholma (NordSydlinjen) via Furusund,3,3,NaN,1,,
2090775,14010000632876421,19:25:00,19:25:00,9022001000552001,16,Norrtälje/Arholma (NordSydlinjen) via Furusund,3,3,NaN,1,,
2090776,14010000632876421,20:15:00,20:15:00,9022001000554001,17,Norrtälje/Arholma (NordSydlinjen) via Furusund,1,3,NaN,1,,
2090777,14010000631915168,15:30:00,15:30:00,9022001000541001,1,Räfsnäs,1,1,NaN,1,,


In [18]:
df_stop["stop_name"].unique()

array(['', 'Fridhemsplan', 'Västerbroplan', 'Hornstull', 'Varvsgatan',
       'Zinkensdamm', 'Södra station', 'Rosenlund', 'Eriksdal',
       'Skanstull', 'Gullmarsplan', 'Radiohuset', 'Garnisonen',
       'Banérgatan', 'Värtavägen', 'Musikhögskolan', 'Stadion',
       'Östra station', 'Odengatan', 'Roslagsgatan', 'Stadsbiblioteket',
       'Odenplan', 'Dalagatan', 'S:t Eriksplan', 'Fleminggatan',
       'Valhallavägen', 'Kungsträdgården', 'T-Centralen', 'Rådhuset',
       'Stadshagen', 'Västra skogen', 'Huvudsta', 'Solna strand',
       'Sundbybergs centrum', 'Duvbo', 'Rissne', 'Rinkeby', 'Tensta',
       'Hjulsta', 'Solna centrum', 'Näckrosen', 'Hallonbergen',
       'Kymlinge norrut', 'Kista', 'Husby', 'Akalla', 'Kymlinge söderut',
       'Universitetet', 'Bergshamra', 'Danderyds sjukhus',
       'Mörby centrum', 'Fruängen', 'Västertorp', 'Hägerstensåsen',
       'Telefonplan', 'Midsommarkransen', 'Liljeholmen', 'Mariatorget',
       'Slussen', 'Gamla stan', 'Östermalmstorg', 'Tekni

In [19]:
df_stop.loc[df_stop["stop_name"]=="Östra station","stop_name"] = "Tekniska högskolan"
df_stop.loc[df_stop["stop_name"]=="Solna centrum","stop_name"] = "Solna Centrum"
df_stop.loc[df_stop["stop_name"]=="Solna centrum norra","stop_name"] = "Solna Centrum"
df_stop.loc[df_stop["stop_name"]=="Bergshamra bro","stop_name"] = "Bergshamra"

In [21]:
df_stop[df_stop["stop_name"]!=""].to_csv("save/stops_v2.csv",index=False)

In [20]:
df_stop[df_stop["stop_name"]!=""]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,stop_name,line_name
113555,14010000620087893,07:06:00,07:06:00,9022001010369008,1,Gullmarsplan,3,1,0.00,1,Fridhemsplan,4
113556,14010000620087893,07:07:32,07:07:32,9022001010645001,2,Gullmarsplan,3,3,815.46,0,Västerbroplan,4
113557,14010000620087893,07:11:00,07:11:00,9022001010649001,3,Gullmarsplan,3,3,2365.97,1,Hornstull,4
113558,14010000620087893,07:12:38,07:12:38,9022001010651001,4,Gullmarsplan,3,3,2844.02,0,Varvsgatan,4
113559,14010000620087893,07:14:09,07:14:09,9022001010655001,5,Gullmarsplan,3,3,3298.02,0,Zinkensdamm,4
...,...,...,...,...,...,...,...,...,...,...,...,...
972752,14010000627427625,09:21:16,09:21:16,9022001050233002,10,Brommaplan,3,3,8527.39,0,Solna strand - Huvudstaleden,177
972753,14010000627427625,09:23:42,09:23:42,9022001012157005,11,Brommaplan,3,3,9331.37,0,Voltavägen,177
972754,14010000627427625,09:24:48,09:24:48,9022001012035002,12,Brommaplan,3,3,9720.04,0,Norrbyvägen,177
972755,14010000627427625,09:27:45,09:27:45,9022001012219002,13,Brommaplan,3,3,10712.19,0,Riksbyvägen,177
